## Clifford Fourier Transform for color images

This code corresponds the implementation described in:
"Color object recognition based on a clifford fourier transform". 
Jose Mennesson, Christophe Saint-Jean, and Laurent Mascarilla. 
In book "Guide to Geometric Algebra in Practice". Springer, 2011.

Python code here is an adaptation of Matlab code by the previous authors.

### Mathematical description

In [2]:
from sympy.galgebra import *


ImportError: No module named 'sympy'

### Signal processing implementation

In [3]:
from scipy.misc import imread

## Get Image from file
def image_from_file(filename): 
    return imread(filename)
## Get Image for url
def image_from_url(url):
    from requests import get
    from StringIO import StringIO   
    return image_from_file(StringIO(get(url).content))

ImportError: cannot import name 'imread'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
urls =['https://hal.archives-ouvertes.fr/hal-00332912v2/file/Fhouse_color.png',
       'http://www.cs.cmu.edu/~chuck/lennapg/lena_std.tif']
# just to open image from url
image = image_from_url(urls[0])[:,:,:3]/255.
plt.imshow(image)
plt.show()
 

In [ ]:
import numpy as np

def get_basis(c1, c2 = None):
    c1 /= np.linalg.norm(c1)
    if c2 is None:  
        mu = np.ones(3)
        if np.allclose(np.cross(c1, mu), np.zeros(3)):
            mu = np.array([1.,0.,0.])
        c3 = np.cross(c1,np.cross(mu,c1))
        c3 /= np.linalg.norm(c3)
        c4 = np.cross(c3, c1)
        assert(np.allclose(np.linalg.norm(c4), 1.))
        return np.append(c1,0.), np.array([0,0,0,1.]), np.append(c3,0.), np.append(c4,0.)
    else:
        c3 = np.cross(c1, c2)
        c3 /= np.linalg.norm(c3)
        c2 = -np.cross(c1, c3)
        assert(np.allclose(np.linalg.norm(c2), 1.))
        return np.append(c1,0), np.append(c2,0), np.append(c3,0), np.array([0,0,0,1.])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)

B = get_basis(np.array([1.,0,0]))
fig = plt.figure(figsize = (10, 8))
ax = fig.gca(projection='3d')
ax.set_aspect("equal")
a = Arrow3D([0, B[0][0]], [0, B[0][1]], [0, B[0][2]] , mutation_scale=20, lw=2, arrowstyle="-|>", color="r")
ax.add_artist(a)
a = Arrow3D([0, B[2][0]], [0, B[2][1]], [0, B[2][2]] , mutation_scale=20, lw=2, arrowstyle="-|>", color="g")
ax.add_artist(a)
a = Arrow3D([0, B[3][0]], [0, B[3][1]], [0, B[3][2]] , mutation_scale=20, lw=2, arrowstyle="-|>", color="b")
ax.add_artist(a)
plt.draw()
plt.show()

In [ ]:
def cfft2(image, basis, reconstruction = False):
    h,l,chan = image.shape
    if chan == 4:
        pass
    elif chan == 3:
        image = np.dstack((image, np.zeros((h,l)))) # add a fourth component with 0
    else:
        raise ValueError
    # Computation of two FFT2s on the two projected Image
    output = { 'parallel' : np.fft.fft2(np.dot(image, basis[0]) + 1j* np.dot(image, basis[1])),
              'orthogonal': np.fft.fft2(np.dot(image, basis[2]) + 1j *np.dot(image, basis[3]))}
    if reconstruction is True:
        output['reconstruction'] = np.outer(output['parallel'].real,basis[0])
        output['reconstruction'] += np.outer(output['parallel'].imag, basis[1])
        output['reconstruction'] += np.outer(output['orthogonal'].real, basis[2])
        output['reconstruction'] += np.outer(output['orthogonal'].imag, basis[3])
        output['reconstruction'] = np.reshape(output['reconstruction'],(h,l,4))
    return output


In [ ]:
def icfft2(tf_image, basis):
    if tf_image.has_key('parallel') and tf_image.has_key('orthogonal'):
        h,l = tf_image['parallel'].shape
        _ = np.fft.ifft2(tf_image['parallel'])
        image = np.outer(_.real, basis[0])
        image += np.outer(_.imag, basis[1])
        _ = np.fft.ifft2(tf_image['orthogonal'])
        image += np.outer(_.real, basis[2])
        image += np.outer(_.imag, basis[3])      
    elif fft_image.has_key('reconstruction'):
        _ = tf_image['reconstruction']
        return icfft2({'parallel' : np.dot(_, basis[0]) + 1j* np.dot(_, basis[1]),
                       'orthogonal' : np.dot(_, basis[2]) + 1j* np.dot(_, basis[3])}, basis)
    elif fft_image.shape[2] == 4:
        return icfft2({'reconstruction' : tf_image}, basis)
    else:
        raise ValueError  
    return np.reshape(image, (h,l,4))

In [ ]:
from IPython.html.widgets import interact
from IPython.html import widgets
r_slider = widgets.IntSlider(min=0, max=255, step=1, value=255)
g_slider = widgets.IntSlider(min=0, max=255, step=1, value=0)
b_slider = widgets.IntSlider(min=0, max=255, step=1, value=0)
    
def f(r,g,b):
    basis = get_basis(np.array([r,g,b], dtype=np.double))
    R = cfft2(image, basis)
    IR = icfft2(R, basis)
    fig, ax = plt.subplots(2, 2, figsize=(15,10)) 
    ax[0,0].imshow(image)
    ax[0,0].set_title('Original image')
    ax[1,0].imshow(np.log(1+np.abs(np.fft.fftshift(R['parallel']))), plt.cm.Blues)
    ax[1,0].set_title('Spectrum in first basis (parallel)')
    ax[1,1].imshow(np.log(1+np.abs(np.fft.fftshift(R['orthogonal']))), plt.cm.Blues)
    ax[1,1].set_title('Spectrum in second basis (orthogonal)')
    ax[0,1].imshow(IR[:,:,:3])
    ax[0,1].set_title('Reconstructed image')
              
interact(f, r=r_slider, g=g_slider, b=b_slider)

### Speed considerations

In [ ]:
basis = get_basis(np.array([1,0,0], dtype=np.double))
%timeit icfft2(cfft2(image, basis), basis)